# MongoDB Helper - Episode Livre

Ce module implémente la classe `EpisodeLivre` pour gérer les relations entre épisodes et livres mentionnés dans les avis critiques.

In [1]:
# | default_exp mongo_episode_livre

In [2]:
# | export
from mongo import BaseEntity, get_collection
from config import get_DB_VARS
from mongo_auteur import Auteur
from mongo_livre import Livre
from typing import List, Type, TypeVar, Optional, Dict, Any
from bson import ObjectId
from datetime import datetime
import logging

# Configure logging
logger = logging.getLogger(__name__)

T = TypeVar("T", bound="EpisodeLivre")

In [3]:
# | export
class EpisodeLivre(BaseEntity):
    """
    Classe représentant la relation entre un épisode et un livre mentionné dans les avis critiques.
    
    Cette classe fait le lien entre:
    - Un épisode (via episode_oid)
    - Un livre (via livre_oid) 
    - Un auteur (via auteur_oid)
    - Les métadonnées de l'avis critique (note, section, commentaires)
    """
    
    collection: str = "episode_livres"

    def __init__(self, episode_oid: ObjectId, livre_oid: ObjectId, auteur_oid: ObjectId) -> None:
        """
        Initialise une instance d'EpisodeLivre.
        
        Args:
            episode_oid (ObjectId): L'ObjectId de l'épisode
            livre_oid (ObjectId): L'ObjectId du livre
            auteur_oid (ObjectId): L'ObjectId de l'auteur
        """
        # Créer un nom unique basé sur les ObjectIds
        nom = f"{episode_oid}_{livre_oid}_{auteur_oid}"
        super().__init__(nom, self.collection)
        
        # Attributs principaux
        self.episode_oid = episode_oid
        self.livre_oid = livre_oid
        self.auteur_oid = auteur_oid
        
        # Métadonnées de l'épisode (pour faciliter les recherches)
        self.episode_date = None
        self.episode_title = None
        
        # Métadonnées du livre (pour faciliter les recherches)
        self.livre_titre = None
        self.auteur_nom = None
        self.editeur_nom = None
        
        # Métadonnées de l'avis critique
        self.note_moyenne = None
        self.nb_critiques = None
        self.section = None  # "programme" ou "coups_de_coeur"
        self.commentaire = None
        self.coup_de_coeur = None
        self.chef_doeuvre = None
        
        # Métadonnées de création
        self.created_at = datetime.now()
        self.updated_at = datetime.now()

    def add_episode_metadata(self, episode_date: datetime, episode_title: str) -> None:
        """
        Ajoute les métadonnées de l'épisode.
        
        Args:
            episode_date (datetime): Date de l'épisode
            episode_title (str): Titre de l'épisode
        """
        self.episode_date = episode_date
        self.episode_title = episode_title
        self.updated_at = datetime.now()

    def add_livre_metadata(self, livre_titre: str, auteur_nom: str, editeur_nom: Optional[str] = None) -> None:
        """
        Ajoute les métadonnées du livre.
        
        Args:
            livre_titre (str): Titre du livre
            auteur_nom (str): Nom de l'auteur
            editeur_nom (Optional[str]): Nom de l'éditeur
        """
        self.livre_titre = livre_titre
        self.auteur_nom = auteur_nom
        self.editeur_nom = editeur_nom
        self.updated_at = datetime.now()

    def add_avis_metadata(self, note_moyenne: Optional[float] = None, nb_critiques: Optional[int] = None,
                         section: str = "programme", commentaire: Optional[str] = None,
                         coup_de_coeur: Optional[str] = None, chef_doeuvre: Optional[str] = None) -> None:
        """
        Ajoute les métadonnées de l'avis critique.
        
        Args:
            note_moyenne (Optional[float]): Note moyenne attribuée
            nb_critiques (Optional[int]): Nombre de critiques
            section (str): Section de l'avis ("programme" ou "coups_de_coeur")
            commentaire (Optional[str]): Commentaire détaillé
            coup_de_coeur (Optional[str]): Nom du critique qui en a fait un coup de cœur
            chef_doeuvre (Optional[str]): Nom du critique qui l'a qualifié de chef d'œuvre
        """
        self.note_moyenne = note_moyenne
        self.nb_critiques = nb_critiques
        self.section = section
        self.commentaire = commentaire
        self.coup_de_coeur = coup_de_coeur
        self.chef_doeuvre = chef_doeuvre
        self.updated_at = datetime.now()

    @classmethod
    def from_book_mention(cls, episode_oid: ObjectId, livre_oid: ObjectId, auteur_oid: ObjectId,
                         book_mention: 'BookMention', episode_date: datetime = None,
                         episode_title: str = None) -> 'EpisodeLivre':
        """
        Crée une instance d'EpisodeLivre à partir d'un BookMention.
        
        Args:
            episode_oid (ObjectId): ObjectId de l'épisode
            livre_oid (ObjectId): ObjectId du livre
            auteur_oid (ObjectId): ObjectId de l'auteur
            book_mention (BookMention): Instance de BookMention du parser
            episode_date (datetime): Date de l'épisode
            episode_title (str): Titre de l'épisode
            
        Returns:
            EpisodeLivre: Instance configurée
        """
        instance = cls(episode_oid, livre_oid, auteur_oid)
        
        # Ajouter les métadonnées du livre
        instance.add_livre_metadata(
            livre_titre=book_mention.titre,
            auteur_nom=book_mention.auteur,
            editeur_nom=book_mention.editeur
        )
        
        # Ajouter les métadonnées de l'avis
        instance.add_avis_metadata(
            note_moyenne=book_mention.note,
            nb_critiques=book_mention.nb_critiques,
            section=book_mention.section,
            commentaire=book_mention.commentaire,
            coup_de_coeur=book_mention.coup_de_coeur,
            chef_doeuvre=book_mention.chef_doeuvre
        )
        
        # Ajouter les métadonnées de l'épisode si disponibles
        if episode_date and episode_title:
            instance.add_episode_metadata(episode_date, episode_title)
        
        return instance

    @classmethod
    def find_by_livre(cls: Type[T], livre_oid: ObjectId) -> List[T]:
        """
        Trouve tous les épisodes qui mentionnent un livre donné.
        
        Args:
            livre_oid (ObjectId): ObjectId du livre
            
        Returns:
            List[T]: Liste des instances EpisodeLivre
        """
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=cls.collection
        )
        
        # Rechercher par livre_oid et trier par date d'épisode décroissante
        cursor = collection.find({"livre_oid": livre_oid}).sort("episode_date", -1)
        
        results = []
        for doc in cursor:
            instance = cls._from_document(doc)
            if instance:
                results.append(instance)
        
        return results

    @classmethod
    def find_by_auteur(cls: Type[T], auteur_oid: ObjectId) -> List[T]:
        """
        Trouve tous les épisodes qui mentionnent un auteur donné.
        
        Args:
            auteur_oid (ObjectId): ObjectId de l'auteur
            
        Returns:
            List[T]: Liste des instances EpisodeLivre
        """
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=cls.collection
        )
        
        # Rechercher par auteur_oid et trier par date d'épisode décroissante
        cursor = collection.find({"auteur_oid": auteur_oid}).sort("episode_date", -1)
        
        results = []
        for doc in cursor:
            instance = cls._from_document(doc)
            if instance:
                results.append(instance)
        
        return results

    @classmethod
    def find_by_episode(cls: Type[T], episode_oid: ObjectId) -> List[T]:
        """
        Trouve tous les livres mentionnés dans un épisode donné.
        
        Args:
            episode_oid (ObjectId): ObjectId de l'épisode
            
        Returns:
            List[T]: Liste des instances EpisodeLivre
        """
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=cls.collection
        )
        
        # Rechercher par episode_oid et trier par note décroissante
        cursor = collection.find({"episode_oid": episode_oid}).sort("note_moyenne", -1)
        
        results = []
        for doc in cursor:
            instance = cls._from_document(doc)
            if instance:
                results.append(instance)
        
        return results

    @classmethod
    def get_all_livres_uniques(cls) -> List[Dict[str, Any]]:
        """
        Récupère tous les livres uniques mentionnés dans les avis critiques.
        
        Returns:
            List[Dict[str, Any]]: Liste des livres avec leurs métadonnées
        """
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=cls.collection
        )
        
        # Agrégation pour obtenir les livres uniques avec leurs informations
        pipeline = [
            {
                "$group": {
                    "_id": {
                        "livre_oid": "$livre_oid",
                        "livre_titre": "$livre_titre",
                        "auteur_nom": "$auteur_nom",
                        "editeur_nom": "$editeur_nom"
                    },
                    "nb_episodes": {"$sum": 1},
                    "note_moyenne_globale": {"$avg": "$note_moyenne"},
                    "derniere_mention": {"$max": "$episode_date"},
                    "episodes": {"$push": {
                        "episode_oid": "$episode_oid",
                        "episode_title": "$episode_title",
                        "episode_date": "$episode_date",
                        "note_moyenne": "$note_moyenne",
                        "section": "$section"
                    }}
                }
            },
            {
                "$sort": {"derniere_mention": -1}
            }
        ]
        
        results = list(collection.aggregate(pipeline))
        
        # Reformater les résultats
        livres_uniques = []
        for result in results:
            livre_info = result["_id"]
            livre_info.update({
                "nb_episodes": result["nb_episodes"],
                "note_moyenne_globale": result.get("note_moyenne_globale"),
                "derniere_mention": result.get("derniere_mention"),
                "episodes": result["episodes"]
            })
            livres_uniques.append(livre_info)
        
        return livres_uniques

    @classmethod
    def search_livres(cls, query: str) -> List[Dict[str, Any]]:
        """
        Recherche des livres par titre ou auteur.
        
        Args:
            query (str): Terme de recherche
            
        Returns:
            List[Dict[str, Any]]: Liste des livres correspondants
        """
        if not query or len(query) < 2:
            return []
        
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=cls.collection
        )
        
        # Recherche textuelle sur titre et auteur
        search_filter = {
            "$or": [
                {"livre_titre": {"$regex": query, "$options": "i"}},
                {"auteur_nom": {"$regex": query, "$options": "i"}}
            ]
        }
        
        # Agrégation pour obtenir les livres uniques correspondants
        pipeline = [
            {"$match": search_filter},
            {
                "$group": {
                    "_id": {
                        "livre_oid": "$livre_oid",
                        "livre_titre": "$livre_titre",
                        "auteur_nom": "$auteur_nom",
                        "editeur_nom": "$editeur_nom"
                    },
                    "nb_episodes": {"$sum": 1},
                    "note_moyenne_globale": {"$avg": "$note_moyenne"},
                    "derniere_mention": {"$max": "$episode_date"}
                }
            },
            {
                "$sort": {"derniere_mention": -1}
            },
            {"$limit": 50}  # Limiter les résultats pour les performances
        ]
        
        results = list(collection.aggregate(pipeline))
        
        # Reformater les résultats
        livres_matches = []
        for result in results:
            livre_info = result["_id"]
            livre_info.update({
                "nb_episodes": result["nb_episodes"],
                "note_moyenne_globale": result.get("note_moyenne_globale"),
                "derniere_mention": result.get("derniere_mention")
            })
            livres_matches.append(livre_info)
        
        return livres_matches

    @classmethod
    def _from_document(cls: Type[T], doc: Dict[str, Any]) -> Optional[T]:
        """
        Crée une instance à partir d'un document MongoDB.
        
        Args:
            doc (Dict[str, Any]): Document MongoDB
            
        Returns:
            Optional[T]: Instance EpisodeLivre ou None si erreur
        """
        try:
            instance = cls(
                episode_oid=doc["episode_oid"],
                livre_oid=doc["livre_oid"],
                auteur_oid=doc["auteur_oid"]
            )
            
            # Restaurer tous les attributs
            for key, value in doc.items():
                if hasattr(instance, key) and key not in ["_id"]:
                    setattr(instance, key, value)
            
            return instance
            
        except (KeyError, TypeError) as e:
            logger.warning(f"Erreur lors de la création d'EpisodeLivre depuis document: {e}")
            return None

    def __str__(self) -> str:
        """Représentation string de l'instance."""
        return f"EpisodeLivre({self.auteur_nom}: {self.livre_titre} - Episode: {self.episode_title})"

    def __repr__(self) -> str:
        """Représentation officielle de l'instance."""
        return f"EpisodeLivre(episode_oid={self.episode_oid}, livre_oid={self.livre_oid}, auteur_oid={self.auteur_oid})"

## Tests de la classe EpisodeLivre

Testons les fonctionnalités principales de la classe.

In [4]:
# Test de création d'une instance
if __name__ == "__main__":
    from bson import ObjectId
    
    # Créer des ObjectIds de test
    episode_oid = ObjectId()
    livre_oid = ObjectId()
    auteur_oid = ObjectId()
    
    # Créer une instance
    episode_livre = EpisodeLivre(episode_oid, livre_oid, auteur_oid)
    
    # Ajouter des métadonnées
    episode_livre.add_livre_metadata(
        livre_titre="Le Seigneur des Anneaux",
        auteur_nom="J.R.R. Tolkien",
        editeur_nom="Gallimard"
    )
    
    episode_livre.add_avis_metadata(
        note_moyenne=8.5,
        nb_critiques=3,
        section="programme",
        commentaire="Un chef-d'œuvre de la fantasy"
    )
    
    print(f"Instance créée: {episode_livre}")
    print(f"Titre: {episode_livre.livre_titre}")
    print(f"Note: {episode_livre.note_moyenne}")

Instance créée: EpisodeLivre(J.R.R. Tolkien: Le Seigneur des Anneaux - Episode: None)
Titre: Le Seigneur des Anneaux
Note: 8.5


# extract py

In [6]:
from nbdev.export import nb_export

nb_export("py mongo helper episode livre.ipynb", ".")